This notebook demonstrates how to integrate functionality from Simple Neurite Tracer with Python through pyimagej.  
The context is the computation, rendering and analysis of the convex hulls enclosing all axonal/dendritic terminal nodes.  
It requires that the latest version of SNT is installed in your ImageJ/Fiji environment.  

Python dependencies:
- pyimagej
- numpy
- scipy

In [1]:
import time
import imagej
import numpy as np
from scipy.spatial import ConvexHull

"""Replace with path to your local ImageJ or Fiji installation.
This can be replaced with ij = imagej.init('sc.fiji:fiji')
once the scijava branch is merged to master."""

ij = imagej.init(r'C:\Users\cam\Desktop\Fiji.app', headless=False)
from jnius import autoclass, cast

In [2]:
# import relevant Java classes
HashSet = autoclass('java.util.HashSet')
PointInImage = autoclass('tracing.util.PointInImage')
MouseLightLoader = autoclass('tracing.io.MouseLightLoader')
Tree = autoclass('tracing.Tree')
TreeAnalyzer = autoclass('tracing.analysis.TreeAnalyzer')
Color = autoclass('org.scijava.util.Colors')
Viewer = autoclass('tracing.viewer.Viewer3D')

In [3]:
def obj_hull(obj):
    """Computes and returns convex hull of 
    the given tracing.viewer.OBJMesh object"""
    
    # get Java Collection of OBJ vertices
    java_col = obj.getVertices()
    # convert to Java Array
    java_array = java_col.toArray()
    
    # convert to Python list
    vert_list = []
    for v in java_array:
        vert_list.append([v.x, v.y, v.z]) # get coordinate fields of each PointInImage object
    
    # compute convex hull
    hull = ConvexHull(vert_list)
    
    return hull

In [4]:
def convex_hull(tree, reference_hull=None):
        """Accepts a tracing.Tree object and computes 
        the convex hull, returning the hull vertices 
        as a Java array containing tracing.util.PointInImage 
        objects. Prints spatial measurements by default."""
        
        analyzer = TreeAnalyzer(tree)
        # get the end points from the tree as a Java HashSet
        tips_java_set = analyzer.getTips()
        # convert to Java array to maintain ordering
        tips_java_array = tips_java_set.toArray()

        # convert to Python list
        tips_list = [[t.x, t.y, t.z] for t in tips_java_array] # get coordinate fields of each PointInImage object

        assert len(tips_list) == tips_java_set.size() # check that conversion is successful

        # Find the convex hull of the end points
        hull = ConvexHull(tips_list)
        
        # print spatial measurements
        if reference_hull is not None:
            print("The hull takes up {}% of the brain\'s volume".format(round((hull.volume/reference_hull.volume) * 100, 5)))
        
        print("The volume of the convex hull encapsulating all tip points is {} cubic microns".format(
            round(hull.volume, 5)))
        
        print("The area of the hull is {} square microns".format(round(hull.area, 5)))
        print()
        
        # find original PointInImage objects to pass to Viewer3D
        # less costly than re-constructing a new Java object for each vertex
        verts_java = [tips_java_array[i] for i in hull.vertices]
        verts_java = ij.py.to_java(verts_java)
        
        return verts_java

In [5]:
def run():
    
    # fetch swc from MouseLight database by ID
    loader = MouseLightLoader('AA0100')
    if not loader.isDatabaseAvailable():
        print("Could not connect to ML database", "Error")
        return
    if not loader.idExists():
        print("Somewhow the specified id was not found", "Error")
        return
    
    # get the axon and dendrite compartments of the SWC
    # soma is ignored since it does not contribute to the convex hull
    tree_axon = loader.getTree('axon', None)
    tree_dendrites = loader.getTree('dendrite', None)
    
    # instantiate the Viewer3D object
    viewer = Viewer()
    # load the Allen mouse brain reference OBJ
    # this automatically adds it to viewer scene
    ref_obj = viewer.loadMouseRefBrain()
    # obtain its convex hull
    ref_hull = obj_hull(ref_obj)
    
    # calculate convex hull and return its vertices
    # the vertices are needed to plot the surface
    verts_axon = convex_hull(tree_axon, ref_hull)
    verts_dendrites = convex_hull(tree_dendrites, ref_hull)

    # add both trees and surfaces to the scene 
    # show the result
    viewer.add(tree_axon)
    viewer.add(tree_dendrites)
    viewer.addSurface(verts_axon, Color.RED, 50)
    viewer.addSurface(verts_dendrites, Color.CYAN, 50)
    viewer.show()

In [6]:
run()

The hull takes up 35.35527% of the brain's volume
The volume of the convex hull encapsulating all tip points is 208791575907.3617 cubic microns
The area of the hull is 191199110.73832 square microns

The hull takes up 0.01051% of the brain's volume
The volume of the convex hull encapsulating all tip points is 62095649.0325 cubic microns
The area of the hull is 942390.38006 square microns

